## This section starts us off (api query and processing) 
* This section loops through 5 dates and gets the hrly temp, humidity and wind_speed for 5 days

In [ ]:
'''
The new goal of my project is to: calculate the average temp in manchester over the last 5 days and graph it
'''
# unix time calculator -0> https://www.vultr.com/resources/unix-time-calculator/?convert=date&date_day=27&date_month=10&date_year=2021&date_hour=12&date_minute=00&date_second=00&date_tz_from=UTC&date_tz_to=UTC&date_format=Y-m-d+H%3Ai%3As

import requests, json, datetime, pandas as pd

df_final = pd.DataFrame(columns=['dt', 'temp', 'humidity', 'wind_speed']) # inititalisation of final dataframe
last_5_days = []

for x in range(0,5): # gets current date and time delta in unixtime
    unix_date_time = (datetime.datetime.today() - datetime.timedelta(x)).strftime("%s") 
    last_5_days.append(unix_date_time)




### Manchester dataframe processing

* this gets all the data for the city of manchester, condences it to what we care about and puts it in a dataframe

In [ ]:
df_final_manchester = pd.DataFrame(columns=['dt', 'temp', 'humidity', 'wind_speed']) # inititalisation of final dataframe

for day in last_5_days: # gets the json from the api, processes it into a df and appends it to the final frame. ownly takes 4 columbs
    response_manchester = requests.get("http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=42.9956&lon=-71.4548&dt="+ day +"&appid=fd3ef9b217de69f13fde1bd0325da289")
    
    processed_responce_manchester = response_manchester.json() # gets json
    
    df_raw_manchester = pd.DataFrame(processed_responce_manchester['hourly']) # json -> df from the "hourly" list
    
    df_processed_manchester = df_raw_manchester[["dt","temp", "humidity", "wind_speed"]].copy() # cuts df down to what we want ownly
    
    df_final_manchester = df_final_manchester.append(df_processed_manchester) # appends to the final df

df_final_manchester.insert(loc=len(df_final_manchester.columns), column='city', value='Manchester')

df_final_manchester['id'] = df_final_manchester[['dt', 'temp', 'humidity', 'wind_speed',]].sum(axis=1).map(hash)

df_final_manchester#.shape 

### Concord data processing

* this gets all the data for the city of concord, condences it to what we care about and puts it in a dataframe

In [ ]:
df_final_concord = pd.DataFrame(columns=['dt', 'temp', 'humidity', 'wind_speed']) # inititalisation of final dataframe

for day in last_5_days: # gets the json from the api, processes it into a df and appends it to the final frame. ownly takes 4 columbs
    response_concord = requests.get("http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=43.2081&lon=-71.5376&dt="+ day +"&appid=fd3ef9b217de69f13fde1bd0325da289")
    
    processed_responce_concord = response_concord.json() # gets json
    
    df_raw_concord = pd.DataFrame(processed_responce_concord['hourly']) # json -> df from the "hourly" list
    
    df_processed_concord = df_raw_concord[["dt","temp", "humidity", "wind_speed"]].copy() # cuts df down to what we want ownly
    
    df_final_concord = df_final_concord.append(df_processed_concord) # appends to the final df

df_final_concord.insert(loc=len(df_final_concord.columns), column='city', value='Concord')

df_final_concord['id'] = df_final_concord[['dt', 'temp', 'humidity', 'wind_speed',]].sum(axis=1).map(hash)

df_final_concord #.shape 

## Keen data processing
* this gets all the data for the city of Keen, condences it to what we care about and puts it in a dataframe

In [ ]:
df_final_keen = pd.DataFrame(columns=['dt', 'temp', 'humidity', 'wind_speed']) # inititalisation of final dataframe

for day in last_5_days: # gets the json from the api, processes it into a df and appends it to the final frame. ownly takes 4 columbs
    response_keen = requests.get("http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=41.5929&lon=-75.3738&dt="+ day +"&appid=fd3ef9b217de69f13fde1bd0325da289")
    
    processed_responce_keen = response_keen.json() # gets json
    
    df_raw_keen = pd.DataFrame(processed_responce_keen['hourly']) # json -> df from the "hourly" list
    
    df_processed_keen = df_raw_keen[["dt","temp", "humidity", "wind_speed"]].copy() # cuts df down to what we want ownly
    
    df_final_keen = df_final_keen.append(df_processed_keen) # appends to the final df

df_final_keen.insert(loc=len(df_final_keen.columns), column='city', value='keen')

#df_final_keen['id'].apply(lambda x : df_final_keen[x] = hash)

#df['a'] = df['a'].apply(lambda x: x + 1)

df_final_keen['id'] = df_final_keen[['dt', 'temp', 'humidity', 'wind_speed',]].sum(axis=1).map(hash)



df_final_keen #.shape

## Merge the 3 newly ceated data frames
* This section uses a concat() to mreg the 3 newly made dataframes because all the columbs are the same

In [ ]:
df_big = pd.concat([df_final_manchester, df_final_keen, df_final_concord], axis=0)
df_big

## CSV producuction
* thats it, its just a csv file

In [ ]:
df_big.to_csv(r'/home/anthony/Documents/programing-materals/comp574/DataProject/export_dataframe.csv', index = False, header=True)

## Convert's to ferinheight
* This section take's our concatinated df and converts everything to readable format (ferinheight)
* This also converts our unix timestamp to a readable date format

In [ ]:
df_big.loc[:, "temp"] = df_big["temp"].apply(lambda x: x - 273)
df_big.loc[:, "dt"] = df_big["dt"].apply(lambda x: datetime.datetime.utcfromtimestamp(x).strftime('%m/%d/%Y'))
df_big

#datetime.datetime.utcfromtimestamp(posix_time).strftime('%Y-%m-%dT%H:%M:%SZ')

## This sectoin handles averaging the columbs

#### Avearge Temp for all Cities

In [ ]:
aveage_temp = df_big.groupby('city')['temp'].mean()
aveage_temp # this needs to be converted to regular temp

#### Avearge Temp variance for all Cities

In [ ]:
aveage_temp_var = df_big.groupby('city')['temp'].var()
aveage_temp_var

#### Average humidity for all Cities

In [ ]:
average_humidity = df_big.groupby('city')['humidity'].mean()
average_humidity

#### Average wind speed for all Cities

In [ ]:
aveage_wind_speed = df_big.groupby('city')['wind_speed'].mean()
aveage_wind_speed # This is in MPH already, its just so slow that its under 1 mph

#### city mode
* This section just proves that all the data comes in equily from all cities and that theres no more data fro one city than anouther

In [ ]:
city_mode = df_big['city'].mode()
city_mode

## Graphing section
* thsis section hadles graphing via seborn

In [ ]:
import seaborn as sns

#### Heatmap
* this checks to see if any data is corilated in the dataframe

In [ ]:
graphdf = (df_big.drop(['dt', 'city', 'id'], axis=1).copy())

corr = graphdf.corr()
sns.heatmap(corr)

#### This section crops down our df for gaphing in a bar plot

In [ ]:
bar_graph = (df_big.drop(['dt','id'], axis=1).copy())

#### manchester info
* this section handles getting the average temp for manchester

In [ ]:
manchester_bar = bar_graph.where(bar_graph['city'] == 'Manchester')
manchester_bar.dropna()
mancheser_mean = manchester_bar['temp'].mean()

#### Keen info
* this section handles getting the average temp for Keen

keen_bar = bar_graph.where(bar_graph['city'] == 'keen')
keen_bar.dropna()
keen_mean = keen_bar['temp'].mean()

#### concord info
* this section handles getting the average temp for concord

In [ ]:
Concord_bar = bar_graph.where(bar_graph['city'] == 'Concord')
Concord_bar.dropna()
Concord_mean = Concord_bar['temp'].mean()

#### makeing a graphable df
* this section stiches together our averages and city names to make it graphable 

In [ ]:
data = {'city': ['Manchester', 'keen', 'Concord'], 'average_temp': [mancheser_mean, keen_mean, Concord_mean]}

df = pd.DataFrame(data)

sns.barplot(x = 'city', y = 'average_temp', data=df)